<a href="https://colab.research.google.com/github/Lupama2/IntroCUDA/blob/main/GoogleColab/ICNPG_fusion_numby_cupy_thrust.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%html
<marquee style='width: 100%; color: blue;'><b>ICNPG2023 en Google Colaboratory-Instituto Balseiro</b></marquee>


In [ ]:
#@title si sale un error sobre UTF, probablemente se arregle corriendo esto
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

miremos que version de nvcc tenemos

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


A ver que placa nos toco...

In [ ]:
!nvidia-smi

Fri Apr 28 17:05:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Experimentos con distintos frameworks



Sean dos vectores $x$ e $y$, queremos calcular

$$S=\frac{\sum_{i=0}^n  (x_i+y_i)}{\sqrt{\sum_{i=0}^n (x_i+y_i)^2}}=\frac{\sum_{i=0}^n  z_i}{\sqrt{\sum_{i=0}^n z_i^2}}$$

In [ ]:
#@title numpy
import numpy as np
import time

n = 10000000

# create x and y vectors
x = np.arange(1, n+1, dtype=np.float32)
y = np.arange(n, 0, -1, dtype=np.float32)

start_time = time.time()

# create z vector
z = x + y

# normalize z by its Euclidean norm
norm = np.linalg.norm(z)
#z_norm = z / norm

# sum the elements of the final result
result = np.sum(z)/norm

# print the result and the time taken
print("Result:", result)
print("Time taken:", (time.time() - start_time) * 1000000, "us")

tnumpy = (time.time() - start_time) * 1000000
#print(x,y,z,z_norm)

Result: 3161.5037
Time taken: 21635.53237915039 us


In [ ]:
#@title cupy (solo cambiar np->cp !!!)
import cupy as cp
import time

n = 10000000

# create x and y vectors
x = cp.arange(1, n+1, dtype=cp.float32)
y = cp.arange(n, 0, -1, dtype=cp.float32)
#z = cp.arange(n, 0, -1, dtype=cp.float32)
z= cp.empty(x.shape, dtype=cp.float32)

start_time = time.time()

# create z vector
z = x + y

# normalize z by its Euclidean norm
norm = cp.linalg.norm(z)
#z_norm = z / norm

# sum the elements of the final result
result = cp.sum(z)/norm

# print the result and the time taken
print("Result:", result)
cp.cuda.Stream.null.synchronize()
print("Time taken:", (time.time() - start_time) * 1000000, "us")

tcupy = (time.time() - start_time) * 1000000

#print(x,y,z,z_norm)

Result: 3162.2773
Time taken: 2326.7269134521484 us


In [ ]:
#@title thrust
%%writefile exp.cu
#include <thrust/device_vector.h>
#include <thrust/transform_reduce.h>
#include <thrust/reduce.h>
#include <thrust/iterator/counting_iterator.h>
#include <thrust/functional.h>
#include <thrust/execution_policy.h>
#include <cmath>
#include <iostream>
#include <chrono>

int main()
{
    int n = 10000000;

    // create x and y vectors
    thrust::device_vector<float> x(n);
    thrust::device_vector<float> y(n);
    thrust::sequence(x.begin(), x.end(), 1.f);
    thrust::sequence(y.rbegin(), y.rend(), 1.f);

    // create z vector
    thrust::device_vector<float> z(n);

    auto start_time = std::chrono::high_resolution_clock::now();

    // initialize z = x + y
    thrust::transform(thrust::device, x.begin(), x.end(), y.begin(), z.begin(), thrust::plus<float>());

/*
    thrust::transform(thrust::device, x.begin(), x.end(), y.begin(), z.begin(), [=]__device__ (float a,float b){return a+b;});

    using namespace thrust::placeholders;
    thrust::transform(thrust::device, x.begin(), x.end(), y.begin(), z.begin(), _1+_2);
*/

    // normalize z by its Euclidean norm (transform and reduce fused)
    float norm = std::sqrt(thrust::transform_reduce(thrust::device, z.begin(), z.end(), thrust::square<float>(), 0.f, thrust::plus<float>()));

    // sum the elements of the final result
    float result = thrust::reduce(thrust::device, z.begin(), z.end(), 0.f, thrust::plus<float>())/norm;

    // print the result and the time taken
    std::cout << "Result: " << result << std::endl;

    cudaDeviceSynchronize();
    auto end_time = std::chrono::high_resolution_clock::now();

    auto time_taken = std::chrono::duration_cast<std::chrono::microseconds>(end_time - start_time).count();
    std::cout << "Time taken: " << time_taken << " us" << std::endl;

    return 0;
}


Writing exp.cu


In [ ]:
!nvcc exp.cu; ./a.out

Result: 3162.28
Time taken: 1301 us


In [ ]:
#@title thrust optimized
%%writefile exp2.cu
#include <thrust/device_vector.h>
#include <thrust/transform_reduce.h>
#include <thrust/reduce.h>
#include <thrust/iterator/counting_iterator.h>
#include <thrust/iterator/zip_iterator.h>
#include <thrust/functional.h>
#include <thrust/execution_policy.h>
#include <thrust/tuple.h>
#include <cmath>
#include <iostream>
#include <chrono>

int main()
{
    int n = 10000000;
    //n=4; 

    // create x and y vectors
    thrust::device_vector<float> x(n);
    thrust::device_vector<float> y(n);
    thrust::sequence(x.begin(), x.end(), 1.f);
    thrust::sequence(y.rbegin(), y.rend(), 1.f);

    auto start_time = std::chrono::high_resolution_clock::now();

    auto ambos = 
    thrust::transform_reduce(thrust::device, 
      thrust::make_zip_iterator(
        thrust::make_tuple(x.begin(),y.begin())
      ),
      thrust::make_zip_iterator(
        thrust::make_tuple(x.end(),y.end())
      ),
      [=] __device__ (auto tup)
      {
          float x=thrust::get<0>(tup);
          float y=thrust::get<1>(tup);
          return 
          thrust::make_tuple(
            x+y, 
            (x+y)*(x+y)
          );
      },
      thrust::make_tuple(0.f,0.f),
      [=] __device__ (auto tup1, auto tup2)
      {
          return 
          thrust::make_tuple(
            thrust::get<0>(tup1)+thrust::get<0>(tup2),
            thrust::get<1>(tup1)+thrust::get<1>(tup2)
          );
      }
    );

    float numerador=thrust::get<0>(ambos);
    float denominador=thrust::get<1>(ambos);

    float result=numerador/sqrt(denominador);

    // print the result and the time taken
    std::cout << "Result: " << result << std::endl;

    cudaDeviceSynchronize();
    auto end_time = std::chrono::high_resolution_clock::now();

    auto time_taken = std::chrono::duration_cast<std::chrono::microseconds>(end_time - start_time).count();
    std::cout << "Time taken: " << time_taken << " us" << std::endl;

    return 0;
}


Writing exp2.cu


In [ ]:
!nvcc exp2.cu --extended-lambda; ./a.out

Result: 3162.28
Time taken: 675 us


In [ ]:
print("aceleracion thrust optimized vs thrust =", 1319/661)
print("aceleracion thrust optimized vs cupy =", tcupy/661)
print("aceleracion thrust optimized vs numpy =", tnumpy/661)
print("aceleracion thrust vs cupy =", tcupy/1319)
print("aceleracion cupy vs numpy =", tnumpy/tcupy)

aceleracion thrust optimized vs thrust = 1.9954614220877458
aceleracion thrust optimized vs cupy = 5.015446811508664
aceleracion thrust optimized vs numpy = 34.45491127094956
aceleracion thrust vs cupy = 2.5134270981101037
aceleracion cupy vs numpy = 6.869759079467817
